# List of My Movies

In [67]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from combine import combine


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-01-01 13:16:17.869104


In [99]:
import re
from time import sleep
from collections import OrderedDict
from timeUtils import clock, elapsed
from ioUtils import saveFile, getFile
from fsUtils import setDir, isDir, mkDir, setFile, isFile, setSubFile
from fileUtils import getBaseFilename, getDirname, getFileBasics
from searchUtils import findSubPatternExt, findPatternExt, findExt, findNearest
from strUtils import convertCurrency
from webUtils import getWebData, getHTML
from numpy import repeat
from movieDB import movieDB
from os import getcwd
from os.path import join
from glob import glob
import operator
from movieRenames import manualRenames

##############################################################################################################################
# Combine Movies
##############################################################################################################################
class mymovies(movieDB):
    def __init__(self, basedir=None):
        self.name = "mymovies"
        movieDB.__init__(self, dbdir=self.name)
        self.combine = None
        
    def setCombinedMovies(self, combine):
        self.combine = combine
        
    def getCombinedMovies(self, debug=False):        
        savename = setFile(self.combine.getResultsDir(), "movies.json")
        if not isFile(savename):
            raise ValueErrro("Cannot access {0}".format(savename))
        combinedMovies = getFile(savename)
        if debug:
            print("Found {0} combined movies".format(len(combinedMovies)))
        return combinedMovies
        
    def getMyMovies(self, debug=False): 
        savename = setFile(self.getDataDir(), "mymovies.json")
        if not isFile(savename):
            raise ValueErrro("Cannot access {0}".format(savename))
        mine = getFile(savename)
        if debug:
            print("Found {0} my movies".format(len(mine)))
        return mine
        
    def findMyMovies(self, debug=False):
        movies = glob("/Volumes/*/Movies/*")
        mine   = dict(zip([getBaseFilename(x) for x in movies], movies))
        print("Found {0} movies on my disks".format(len(movies)))
        savename = setFile(self.getDataDir(), "mymovies.json")
        saveFile(idata=mine, ifile=savename, debug=True)
        
        
    def searchMyMovies(self, movie, year, cutoff=0.8):
        moviesMine = self.getMyMovies()
        key = "{0} [{1}]".format(movie, year)
        result     = findNearest(key, moviesMine.keys(), 3, 0.95)
        if len(result) > 0:
            print("{0}  --->  {1}".format(movie, result))
            return
        else:
            key = "{0} [{1}]".format(movie, year)
            result = findNearest(key, moviesMine.keys(), 3, cutoff)
            if len(result) > 0:
                print("{0}  --->  {1}".format(movie, result))
                return
        print("No match found")
        return
        
        
        
    def mergeMovies(self, toFile=False):
        moviesCombined = self.getCombinedMovies()
        moviesMine     = self.getMyMovies()
        renames        = {}
        filename = setFile(self.getResultsDir(), "mymovies.dat")

        if toFile:
            print("{0: <6}{1: <3}{2: <100}{3: <20}".format("Year", u'\u2713', "Movie", "Source"), file=open(filename, "w"))
        else:
            print("{0: <6}{1: <3}{2: <100}{3: <20}".format("Year", u'\u2713', "Movie", "Source"))

        pYear = None
        for key,name in moviesCombined.items():
            movie = key[:-7]
            year  = key[-5:-1]

            location = moviesMine.get(key)
            if location is None:
                result = findNearest(key, moviesMine.keys(), 1, 0.98)
                if len(result) > 0:
                    location = u'\u2248'
                    myMovie  = moviesMine[result[0]]
                    fileInfo = getFileBasics(myMovie)
                    myMovieYear = fileInfo[1][-5:-1]
                    if abs(int(year) - int(myMovieYear)) <= 0:
                        renames[myMovie] = join(fileInfo[0], "".join([key, fileInfo[2]]))
                else:
                    result = findNearest(key, moviesMine.keys(), 1, 0.90)
                    if len(result) > 0:
                        location = u'\u2047'
                        myMovie  = moviesMine[result[0]]
                        fileInfo = getFileBasics(myMovie)
                        myMovieYear = fileInfo[1][-5:-1]
                        if abs(int(year) - int(myMovieYear)) <= 0:
                            renames[myMovie] = join(fileInfo[0], "".join([key, fileInfo[2]]))
                    else:
                        location = ""
            else:
                location = u'\u2713'
                
            
            #print(pYear,year)
            if pYear is not None and pYear != year:
                print("")
            pYear = year
                
            if toFile:
                print("{0: <6}{1: <3}{2: <100}{3: <20}".format(year, location, movie, name), file=open(filename, "a"))
            else:
                print("{0: <6}{1: <3}{2: <100}{3: <20}".format(year, location, movie, name))

                
                
        print("\n\n\n")
        for key,rename in renames.items():
            if key != rename:
                print("moveFile(src=\"{0}\", dst=\"{1}\", debug=True)\n".format(key, rename))

In [108]:
mine = mymovies()
mine.setCombinedMovies(combine())

In [107]:
mine.findMyMovies()

Found 4930 movies on my disks
Saving data to /Users/tgadfort/Documents/code/movies/mymovies/data/mymovies.json
Saved data to /Users/tgadfort/Documents/code/movies/mymovies/data/mymovies.json
  --> This file is 400.2kB.


In [43]:
mine.searchMyMovies("The Rose", 2011, cutoff=0.7)

The Rose  --->  ['The Rite [2011]', 'The Rover [2014]', 'The Roommate [2011]']


In [109]:
mine.mergeMovies()

Year  ✓  Movie                                                                                               Source              
1915     The Birth Of A Nation                                                                               Rotten Tomatoes     

1920     The Cabinet Of Dr. Caligari (Das Cabinet des Dr. Caligari)                                          Rotten Tomatoes     

1922     Nosferatu, A Symphony Of Horror (Nosferatu, eine Symphonie des Grauens) (Nosferatu The Vampire)     Rotten Tomatoes     
1922     Robin Hood                                                                                          Wiki Film           
1922     Oliver Twist                                                                                        Wiki Film           
1922     Blood And Sand                                                                                      Wiki Film           
1922     Manslaughter                                                                   

1930     The Blue Angel                                                                                      Ultimate Movie Rankings
1930     The Divorcee                                                                                        Ultimate Movie Rankings
1930     Tom Sawyer                                                                                          Ultimate Movie Rankings
1930  ⁇  The Dawn Patrol                                                                                     Ultimate Movie Rankings
1930     Ingagi                                                                                              Wiki Film           
1930     Whoopee!                                                                                            Wiki Film           
1930     Check And Double Check                                                                              Wiki Film           
1930     Min And Bill                                                         

1934     The Affairs Of Cellini                                                                              Oscar               
1934     Of Human Bondage                                                                                    Oscar               
1934     The Barretts Of Wimpole Street                                                                      Oscar               
1934     The Merry Widow                                                                                     Oscar               
1934     Eskimo                                                                                              Oscar               
1934     La Cucaracha                                                                                        Oscar               
1934     City Of Wax                                                                                         Oscar               
1934     Manhattan Melodrama                                                              

1936     The General Died At Dawn                                                                            Oscar               
1936     Pigskin Parade                                                                                      Oscar               
1936     The Gorgeous Hussy, Rachel Jackson                                                                  Oscar               
1936     These Three                                                                                         Oscar               
1936     Give Me Liberty                                                                                     Oscar               
1936     Bored Of Education                                                                                  Oscar               
1936     The Public Pays                                                                                     Oscar               
1936     The Country Cousin                                                               

1938     La Grande illusion (Grand Illusion)                                                                 Rotten Tomatoes     
1938  ✓  Bringing Up Baby                                                                                    Rotten Tomatoes     
1938     Test Pilot                                                                                          Wiki Film           
1938     Sweethearts                                                                                         Wiki Film           
1938     Happy Landing                                                                                       Wiki Film           
1938     Out West With The Hardys                                                                            Wiki Film           
1938     Too Hot To Handle                                                                                   Wiki Film           
1938     That Certain Age                                                                 

...

2016     None                                                                                                Box Office          
2016  ✓  Ride Along 2                                                                                        Box Office          
2016  ✓  Don't Breathe                                                                                       Box Office          
2016  ⁇  Sisters                                                                                             Box Office          
2016  ✓  Miss Peregrine's Home For Peculiar Children                                                         Box Office          
2016  ⁇  Alvin And The Chipmunks The Road Chip                                                               Box Office          
2016  ✓  The Accountant                                                                                      Box Office          
2016  ✓  Teenage Mutant Ninja Turtles Out Of The Shadows                                  

2016  ⁇  Woodlawn                                                                                            Box Office          
2016  ⁇  Burnt                                                                                               Box Office          
2016     Hillary's America The Secret History Of The Democratic Party                                        Box Office          
2016     Triple 9                                                                                            Box Office          
2016  ✓  The Light Between Oceans                                                                            Box Office          
2016  ⁇  The 33                                                                                              Box Office          
2016  ✓  Fifty Shades Of Black                                                                               Box Office          
2016     No Manches Frida                                                                 

2017     Quest                                                                                               Rotten Tomatoes     
2017     Dina                                                                                                Rotten Tomatoes     
2017     Trophy                                                                                              Rotten Tomatoes     
2017     The Last Laugh                                                                                      Rotten Tomatoes     
2017     Dolores                                                                                             Rotten Tomatoes     
2017     The Survivalist                                                                                     Rotten Tomatoes     
2017     Your Name. (Kimi No Na Wa.)                                                                         Rotten Tomatoes     
2017     Lucky                                                                            

2017     Annabelle Creation                                                                                  Box Office          
2017  ⁇  Daddy's Home 2                                                                                      Box Office          
2017  ⁇  Murder On The Orient Express                                                                        Box Office          
2017     Kingsman: The Golden Circle                                                                         Box Office          
2017  ⁇  Passengers                                                                                          Box Office          
2017     None                                                                                                Box Office          
2017  ⁇  John Wick Chapter Two                                                                               Box Office          
2017  ≈  Miss Peregrine's Home For Peculiar Children                                      

2017     Victoria And Abdul                                                                                  Box Office          
2017  ✓  Rough Night                                                                                         Box Office          
2017     My Little Pony: The Movie                                                                           Box Office          
2017     Leap!                                                                                               Box Office          
2017  ✓  Sleepless                                                                                           Box Office          
2017  ⁇  Diary Of A Wimpy Kid The Long Haul                                                                  Box Office          
2017  ✓  The Circle                                                                                          Box Office          
2017     Baahubali 2 The Conclusion                                                       

In [66]:
mine.searchMyMovies("Y Tu Mama Tambien", 1985, cutoff=0.7)

Y Tu Mama Tambien  --->  ['Y Tu Mamá También [2001]']


In [86]:
from fsUtils import moveFile
moveFile(src="/Volumes/Save/Movies/What's Up, Doc [1972].mp4", dst="/Volumes/Save/Movies/What's Up, Doc? [1972].mp4")

In [93]:

moveFile(src="/Volumes/Save/Movies/Brief Encounter [1946].mkv", dst="/Volumes/Save/Movies/Brief Encounter [1945].mkv")

moveFile(src="/Volumes/Seagate/Movies/Great Expectations [1947].mkv", dst="/Volumes/Seagate/Movies/Great Expectations [1946].mkv")

In [94]:
moveFile(src="/Volumes/Seagate/Movies/Star Trek II The Wrath Of Khan [1982].mp4", dst="/Volumes/Seagate/Movies/Star Trek II: The Wrath Of Khan [1982].mp4")

In [98]:

moveFile(src="/Volumes/Seagate/Movies/Dirty Harry 2 Magnum Force [1973].mp4", dst="/Volumes/Seagate/Movies/Dirty Harry 2: Magnum Force [1973].mp4")

moveFile(src="/Volumes/Seagate/Movies/The Godfather Part II [1974].mp4", dst="/Volumes/Seagate/Movies/The Godfather: Part II [1974].mp4")

moveFile(src="/Volumes/Seagate/Movies/Dirty Harry 3 The Enforcer [1976].mp4", dst="/Volumes/Seagate/Movies/Dirty Harry 3: The Enforcer [1976].mp4")

In [103]:

moveFile(src="/Volumes/Blue/Movies/Marriage Italian-Style [1964].mkv", dst="/Volumes/Blue/Movies/Marriage Italian Style [1964].mkv")

moveFile(src="/Volumes/Blue/Movies/Seance On A Wet Afternoon [1964].mkv", dst="/Volumes/Blue/Movies/Séance On A Wet Afternoon [1964].mkv")

moveFile(src="/Volumes/Seagate/Movies/Whats New Pussycat [1965].mp4", dst="/Volumes/Seagate/Movies/What's New Pussycat? [1965].mp4")

moveFile(src="/Volumes/Save/Movies/The Russians Are Coming, The Russians Are Coming [1966].mp4", dst="/Volumes/Save/Movies/The Russians Are Coming! The Russians Are Coming! [1966].mp4")

moveFile(src="/Volumes/Save/Movies/What's Up, Doc? [1972].mp4", dst="/Volumes/Save/Movies/What's Up Doc? [1972].mp4")

In [106]:
moveFile(src="/Volumes/Blue/Movies/Calendar Girl [2004].avi", dst="/Volumes/Blue/Movies/Calendar Girls [2004].avi", debug=True)

Moving /Volumes/Blue/Movies/Calendar Girl [2004].avi to /Volumes/Blue/Movies/Calendar Girls [2004].avi
  Not moving because /Volumes/Blue/Movies/Calendar Girls [2004].avi exists.


In [110]:

moveFile(src="/Volumes/Seagate/Movies/Kings Row [1942].avi", dst="/Volumes/Seagate/Movies/King's Row [1942].avi", debug=True)

Moving /Volumes/Seagate/Movies/Kings Row [1942].avi to /Volumes/Seagate/Movies/King's Row [1942].avi
